# LBP

In [2]:
from skimage.feature import hog
from skimage.feature import local_binary_pattern
import cv2 
import os
from sklearn import svm
import pandas as pd
from sklearn.metrics import classification_report,accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
import joblib

In [3]:
# method to create data set
def create_dataset(old_path, young_path, adult_path):
    
    # list to store extracted features of an image
    features = []
    
    # list to store class label, 1 for live, 0 for spoof
    labels = []    
    
    radius = 3
    
    # number of neighbors to consider for LBP
    n_points = 8 * radius 
    
    # sampling type for LBP
    METHOD = 'uniform'       
    
    path_array = [old_path, young_path, adult_path]
    
    for path in path_array:
        
        # storing all images in a folder in a list 'files'
        files = os.listdir(path) 
        
        # loop through the images in the folder
        for img in files:
            
            
            # reading the image in grayscale using cv2
            image = cv2.imread(os.path.join(path + img), cv2.IMREAD_GRAYSCALE)
            
            
            # resizing the image so all images are of same size
            
            resized_img = cv2.resize(image, (300, 300))
            
            # Extracting features of an image using LBP
               
            lbp = local_binary_pattern(resized_img, n_points, radius, METHOD)
                
                 # Converting into 1-D array
            fd=lbp.flatten()
                
            
                
                # label 1 for adult images, 0 for YOung images, 2 for old
            class_identifier = 1
            if 'Young' in path:
                class_identifier = 0
            elif 'Old' in path:
                class_identifier = 2
                
             # appending exracted features to the list
            features.append(fd) 
            
            #adding corresponding class label to the list
            labels.append(class_identifier)  
                      
    return features,labels

In [4]:
# variables for differet folders
training_old_path = "D:\\College\\Third Year\\Computer Vision\\cp2\\Datasets\\Common Dataset\\Train\\Old\\"
training_young_path = "D:\\College\\Third Year\\Computer Vision\\cp2\\Datasets\\Common Dataset\\Train\\Young\\"
training_adult_path = "D:\\College\\Third Year\\Computer Vision\\cp2\\Datasets\\Common Dataset\\Train\\Adult\\"
testing_old_path = "D:\\College\\Third Year\\Computer Vision\\cp2\\Datasets\\Common Dataset\\Test\\Old\\"
testing_young_path = "D:\\College\\Third Year\\Computer Vision\\cp2\\Datasets\\Common Dataset\\Test\\Young\\"
testing_adult_path = "D:\\College\\Third Year\\Computer Vision\\cp2\\Datasets\\Common Dataset\\Test\\Adult\\"

In [5]:
# LBP 
# Training and testing datasets

x_train_data,y_train = create_dataset(training_old_path, training_young_path, training_adult_path)

print('training data captured')

x_test,y_test = create_dataset(testing_old_path,testing_young_path, testing_adult_path)
print('testing data captured')



training data captured
testing data captured


In [9]:
# Create and fit the model

lbp_clf = svm.SVC()
lbp_clf.fit(x_train_data,y_train)

# Predict on the test features, print the results

lbp_y_pred = lbp_clf.predict(x_test)

In [10]:
#SVM Linear
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score
from sklearn import metrics

print("Accuracy:  \t\t"+str(accuracy_score(y_test, lbp_y_pred)))
print('Balanced accuracy: \t', balanced_accuracy_score(y_test, lbp_y_pred))
print('Precision accuracy: \t', precision_score(y_test, lbp_y_pred,average='macro'))
print("Recall Score: \t\t",metrics.recall_score(y_test, lbp_y_pred,average='weighted'))
print("F1 Score: \t\t",metrics.f1_score(y_test, lbp_y_pred,average='micro'))
print('\nConfusion Matrix: \n',confusion_matrix(y_test, lbp_y_pred))


Accuracy:  		0.7055961070559611
Balanced accuracy: 	 0.36604372566636717
Precision accuracy: 	 0.6251082251082251
Recall Score: 		 0.7055961070559611
F1 Score: 		 0.7055961070559611

Confusion Matrix: 
 [[564   0  36]
 [149  10   0]
 [ 57   0   6]]


In [11]:
#Decision Tree Classifier
from sklearn.metrics import accuracy_score
#Assign model with Decision Tree classifier
model_dt = DecisionTreeClassifier(max_depth=13)
model_dt.fit(x_train_data, y_train)
joblib.dump(model_dt,"model_dt")

#predicting the traget variable using testing variables
y_pred1 = model_dt.predict(x_test)
#Results
print("Decision Tree Results")
print("Decision Tree Accuracy: ",accuracy_score(y_test, y_pred1)*100,"%")
print("Train Accuracy:",model_dt.score(x_train_data, y_train))
print("Test Accuracy:",model_dt.score(x_test, y_test))
print("Accuracy:  \t\t"+str(accuracy_score(y_test, y_pred1)))
print('Balanced accuracy: \t', balanced_accuracy_score(y_test, y_pred1))
print('Precision accuracy: \t', precision_score(y_test, y_pred1,average='macro'))
print("Recall Score: \t\t",metrics.recall_score(y_test, y_pred1,average='weighted'))
print("F1 Score: \t\t",metrics.f1_score(y_test, y_pred1,average='micro'))
print('\nConfusion Matrix: \n',confusion_matrix(y_test, y_pred1))


Decision Tree Results
Decision Tree Accuracy:  70.07299270072993 %
Train Accuracy: 0.9827956989247312
Test Accuracy: 0.7007299270072993
Accuracy:  		0.7007299270072993
Balanced accuracy: 	 0.6404312668463611
Precision accuracy: 	 0.5766448611741294
Recall Score: 		 0.7007299270072993
F1 Score: 		 0.7007299270072993

Confusion Matrix: 
 [[480  92  28]
 [112  42   5]
 [  6   3  54]]


In [6]:
#Random Forest classifier
model_rf = RandomForestClassifier(n_estimators = 50)

model_rf.fit(x_train_data, y_train)
joblib.dump(model_rf,"model_rf")

y_pred2 = model_rf.predict(x_test)
print("Random Forest Clasifier")
print("Train Accuracy:",model_rf.score(x_train_data, y_train))
print("Test Accuracy:",model_rf.score(x_test, y_test))
print("Accuracy:  \t\t"+str(accuracy_score(y_test, y_pred2)))
print('Balanced accuracy: \t', balanced_accuracy_score(y_test, y_pred2))
print('Precision accuracy: \t', precision_score(y_test, y_pred2,average='macro'))
print("Recall Score: \t\t",metrics.recall_score(y_test, y_pred2,average='weighted'))
print("F1 Score: \t\t",metrics.f1_score(y_test, y_pred2,average='micro'))
print('\nConfusion Matrix: \n',confusion_matrix(y_test, y_pred2))

Random Forest Clasifier
Train Accuracy: 1.0
Test Accuracy: 0.8090024330900243
Accuracy:  		0.8090024330900243


NameError: name 'balanced_accuracy_score' is not defined

In [13]:
#KNN classifier
model_knn = KNeighborsClassifier(n_neighbors = 5)
model_knn.fit(x_train_data, y_train)
#joblib.dump(model_knn,"model_knn")
y_pred3 = model_knn.predict(x_test)
print("KNN")
print("Train Accuracy:",model_knn.score(x_train_data, y_train))
print("Test Accuracy:",model_knn.score(x_test, y_test))
print("Accuracy:  \t\t"+str(accuracy_score(y_test, y_pred3)))
print('Balanced accuracy: \t', balanced_accuracy_score(y_test, y_pred3))
print('Precision accuracy: \t', precision_score(y_test, y_pred3,average='macro'))
print("Recall Score: \t\t",metrics.recall_score(y_test, y_pred3,average='weighted'))
print("F1 Score: \t\t",metrics.f1_score(y_test, y_pred3,average='micro'))
print('\nConfusion Matrix: \n',confusion_matrix(y_test, y_pred3))

C:\Users\omkar\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


KNN


C:\Users\omkar\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Train Accuracy: 0.8311827956989247
Test Accuracy: 0.5693430656934306
Accuracy:  		0.5693430656934306
Balanced accuracy: 	 0.44126235399820307
Precision accuracy: 	 0.34888135981261903
Recall Score: 		 0.5693430656934306
F1 Score: 		 0.5693430656934306

Confusion Matrix: 
 [[425  64 111]
 [151   7   1]
 [ 27   0  36]]


C:\Users\omkar\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [14]:
#xgboost classifier
model_xgboost = XGBClassifier().fit(x_train_data, y_train)
joblib.dump(model_xgboost,"model_xgboost")
y_pred5 = model_xgboost.predict(x_test)
print("XGboost")
print("Train Accuracy:",model_xgboost.score(x_train_data, y_train))
print("Test Accuracy:",model_xgboost.score(x_test, y_test))
print("Accuracy:  \t\t"+str(accuracy_score(y_test, y_pred5)))
print('Balanced accuracy: \t', balanced_accuracy_score(y_test, y_pred5))
print('Precision accuracy: \t', precision_score(y_test, y_pred5,average='macro'))
print("Recall Score: \t\t",metrics.recall_score(y_test, y_pred5,average='weighted'))
print("F1 Score: \t\t",metrics.f1_score(y_test, y_pred5,average='micro'))
print('\nConfusion Matrix: \n',confusion_matrix(y_test, y_pred5))

XGboost
Train Accuracy: 1.0
Test Accuracy: 0.7944038929440389
Accuracy:  		0.7944038929440389
Balanced accuracy: 	 0.6985050414295698
Precision accuracy: 	 0.7368970592367785
Recall Score: 		 0.7944038929440389
F1 Score: 		 0.7944038929440389

Confusion Matrix: 
 [[545  44  11]
 [104  55   0]
 [ 10   0  53]]
